In [1]:
from bd3client.CentralService import CentralService
from bd3client.Sensor import Sensor
from bd3client.Timeseries import Timeseries
import os
import json
import arrow
import pandas as pd

In [4]:

with open("config/bd3config.json", "r") as fp:
    configs = json.load(fp)
    hostname = configs['hostname']
    valid_naes = configs['valid_naes']
    base_dir = configs['base_dir']
    begin_time = arrow.get(configs['begin_time']).timestamp
    end_time = arrow.get(configs['end_time']).timestamp
with open("config/bd3credential.json", "r") as fp:
    cred = json.load(fp)
    cid = cred['cid']
    ckey = cred['ckey']
cs = CentralService(hostname, cid, ckey)
sensor_api = Sensor(cs)
ts_api = Timeseries(cs)

with open("metadata/bacnet_devices.json", "r") as fp:
    bacnet_devices = json.load(fp)

In [5]:
uuid_dict = dict()
i = 0
for nae_name in valid_naes:
    objs = bacnet_devices[nae_name]['objs']
    for obj in objs:
        if obj['data_type'] not in [0,1,2,3,4,5,13,14,19]:
            continue        
        if i%500==0:
            print i
        i += 1
        props = obj['props']
        tag_val_dict = {
            'BACNet_DataType': obj['data_type'],
            'BACNet_InstanceNumber':obj['props']['instance'],
            'NAENum':nae_name
            }
        srcid = str(nae_name) + "_" + str(obj['data_type']) + "_" + str(obj['props']['instance'])
        try:
            uuid = sensor_api.search(tag_val_dict)['result'][0]['name']
            uuid_dict[srcid] = uuid
        except:
            print '----------------'
            print srcid
            print sensor_api.search(tag_val_dict)        

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500


In [6]:
header = ['value']
index_label="time"
for srcid, uuid in uuid_dict.items():
    try:
        ts = ts_api.getTimeseriesDataPoints(uuid, begin_time, end_time)
        time_list = [arrow.get(row[0]) for row in ts['data']['series'][0]['values']]
        value_list = [row[2] for row in ts['data']['series'][0]['values']]
        filename = base_dir + "/" + srcid + ".csv"
        pd.Series(index=time_list, data=value_list).to_csv(filename, header=header, index_label=index_label)
    except:
        print srcid, uuid
        print ts_api.getTimeseriesDataPoints(uuid, begin_time, end_time)

523_2_3008181 3febbbb6-2042-4047-bd7c-62fed9cbdaa3
{u'data': {}, u'success': u'True'}
523_2_3008180 3ab69a1c-58ea-49a5-bc95-287133078572
{u'data': {}, u'success': u'True'}
523_2_3010581 eee89371-12bf-4da7-85c7-147a6a911fa0
{u'data': {}, u'success': u'True'}
572_3_3017087 ec0a3735-95a4-44db-8c4d-f9062854a278
{u'data': {}, u'success': u'True'}
572_3_3017086 9007894e-b309-4c2d-a3e3-977d7da33605
{u'data': {}, u'success': u'True'}
668_0_3000013 269f85b2-e934-4797-8255-554b59c28c08
{u'data': {}, u'success': u'True'}
520_1_3002195 23ea0055-d62c-48b2-9c33-d77a860c7b21
{u'data': {}, u'success': u'True'}
523_2_3008174 fc3faa50-6792-4153-a903-a92c42a3dcc0
{u'data': {}, u'success': u'True'}
523_2_3008175 ffd040a2-c22a-4732-bd99-94b4f8c924b9
{u'data': {}, u'success': u'True'}
523_2_3008176 5e694b3e-9d67-40f7-b61c-1a76caaa94ca
{u'data': {}, u'success': u'True'}
